In [7]:
import pandas as pd
import numpy as np

# Simple signature implementation as alternative to esig/iisignature
def compute_path_signature(path, depth=3):
    """
    Compute a simplified path signature using basic features.
    This is a simplified alternative to the full signature method.
    """
    if len(path) < 2:
        return np.array([])
    
    # Basic path features
    features = []
    
    # Level 1: Basic statistics
    time_vals = path[:, 0]
    voltage_vals = path[:, 1]
    
    features.extend([
        np.mean(time_vals), np.std(time_vals),
        np.mean(voltage_vals), np.std(voltage_vals),
        np.max(voltage_vals), np.min(voltage_vals)
    ])
    
    # Level 2: Differences and trends  
    if len(path) > 1:
        dt = np.diff(time_vals)
        dv = np.diff(voltage_vals)
        features.extend([
            np.mean(dt), np.std(dt),
            np.mean(dv), np.std(dv),
            np.sum(dv > 0), np.sum(dv < 0)  # trend indicators
        ])
    
    # Level 3: Second order differences
    if len(path) > 2:
        d2v = np.diff(dv)
        features.extend([
            np.mean(d2v), np.std(d2v)
        ])
    
    return np.array(features)

In [8]:
file_path = "./datacsv/processed_battery_discharge_data_filtered.xlsx"
xls = pd.ExcelFile(file_path, engine='openpyxl')

In [9]:
# Set the signature truncation depth
signature_level = 3
result_data = []

for sheet_name in xls.sheet_names:
    df = xls.parse(sheet_name)
    if 'cycle' in df.columns and 'time' in df.columns and 'voltage' in df.columns and 'capacity' in df.columns:
        for cycle, group in df.groupby('cycle'):
            path = group[['time', 'voltage']].to_numpy()
            if len(path) < 2:
                continue
            try:
                # Using our custom signature implementation
                sig = compute_path_signature(path, signature_level)
            except:
                continue
            cap = group['capacity'].iloc[0]
            last_time = group['time'].iloc[-1] 
            last_voltage = group['voltage'].iloc[-1]

            label = last_voltage == 0  

            record = {
                'sheet_cycle': f'{sheet_name}_{cycle}',
                'capacity': cap,
                'event': int(label), 
                'time': last_time
            }
            for idx, val in enumerate(sig):
                record[f's_{idx}'] = val
            result_data.append(record)

df_result = pd.DataFrame(result_data)
df_result.to_csv("./datacsv/battery_signature_features.csv", index=False)

In [10]:
# Corrected file path
file_path = './datacsv/battery_signature_features.csv'  # Make sure this path is correct

# Load the CSV file
df = pd.read_csv(file_path)

# Show the first 5 rows
print("First 5 rows of the CSV file:")
print(df.head())

First 5 rows of the CSV file:
         sheet_cycle  capacity  event      time          s_0         s_1  \
0  B0036_discharge_1  1.001983      0  1819.406   907.120825  525.862745   
1  B0036_discharge_2  1.801101      0  2512.562  1249.074402  724.908133   
2  B0036_discharge_3  1.804423      0  2512.391  1248.555688  724.111883   
3  B0036_discharge_4  1.804389      0  2516.906  1250.539189  725.427093   
4  B0036_discharge_5  1.798925      0  2511.860  1247.618805  724.114657   

        s_2       s_3    s_4    s_5       s_6       s_7       s_8       s_9  \
0  2.305825  0.180385  3.863  1.690  9.628074  0.161506 -0.011559  0.097224   
1  2.530100  0.166274  4.193  2.337  9.627823  0.183555 -0.007138  0.081969   
2  2.541069  0.166894  4.199  2.344  9.664216  0.193810 -0.007162  0.082496   
3  2.541085  0.167117  4.200  2.345  9.719236  0.202361 -0.007190  0.082655   
4  2.541455  0.169054  4.200  2.342  9.775574  0.207923 -0.007258  0.082474   

   s_10   s_11      s_12      s_13  
0